In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import tensorflow as tf 

from baseline import Baseline
from system_t import System_T

# Load Fashion-MNIST dataset from keras

In [2]:
fashion = tf.keras.datasets.fashion_mnist
(fashion_train_images,fashion_train_labels), (fashion_test_images, fashion_test_labels) = fashion.load_data()

fashion_train_labels = fashion_train_labels
fashion_test_labels = fashion_test_labels
fashion_train_images = fashion_train_images.reshape((60000, 28, 28, 1)) / 255.0
fashion_test_images = fashion_test_images.reshape((10000, 28, 28, 1)) / 255.0

train_images = fashion_train_images
train_labels = fashion_train_labels

# Bad for Water filling: a pathological setting for Water filling

In [3]:
def shuffle(data, label):
    shuffle = np.arange(len(data))
    np.random.shuffle(shuffle)
    data = data[shuffle]
    label = label[shuffle]
    return data, label

num_class = len(np.unique(train_labels))
print("Number of slices : %d" % num_class)

y_train_one_hot = tf.keras.utils.to_categorical(train_labels)
mixed_data = (train_images, y_train_one_hot)

initial_data_array = []
val_data_dict = []
add_data_dict = []
    
val_data_num = 500

for i in range(num_class):
    if i in [0, 2, 4, 6]:
        data_num = 300
    else:
        data_num = 100
    initial_data_array.append(data_num)
    idx = np.argmax(mixed_data[1], axis=1) == i

    val_data_dict.append((mixed_data[0][idx][data_num:data_num+val_data_num], mixed_data[1][idx][data_num:data_num+val_data_num]))
    add_data_dict.append((mixed_data[0][idx][data_num+val_data_num:], mixed_data[1][idx][data_num+val_data_num:]))

    if i == 0:
        train_data = mixed_data[0][idx][:data_num]
        train_label = mixed_data[1][idx][:data_num]
        val_data = mixed_data[0][idx][data_num:data_num+val_data_num]
        val_label = mixed_data[1][idx][data_num:data_num+val_data_num]
    else:
        train_data = np.concatenate((train_data, mixed_data[0][idx][:data_num]), axis=0)
        train_label = np.concatenate((train_label, mixed_data[1][idx][:data_num]), axis=0) 
        val_data = np.concatenate((val_data, mixed_data[0][idx][data_num:data_num+val_data_num]), axis=0)
        val_label = np.concatenate((val_label, mixed_data[1][idx][data_num:data_num+val_data_num]), axis=0)   
    
train_data, train_label = shuffle(train_data, train_label)
print(train_data.shape, train_label.shape, num_class)

Number of slices : 10
(1800, 28, 28, 1) (1800, 10) 10


# Define slices

In [4]:
slice_desc = []
a = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

for i in range(num_class):
    slice_desc.append('Slice: %s' % (a[i]))
    print('Slice: %s, Initial size: %s' % (a[i], initial_data_array[i]))

Slice: T-shirt/top, Initial size: 300
Slice: Trouser, Initial size: 100
Slice: Pullover, Initial size: 300
Slice: Dress, Initial size: 100
Slice: Coat, Initial size: 300
Slice: Sandal, Initial size: 100
Slice: Shirt, Initial size: 300
Slice: Sneaker, Initial size: 100
Slice: Bag, Initial size: 100
Slice: Ankle boot, Initial size: 100


# Original ( with no data acquisition ) 

In [5]:
cost_func = [1] * num_class
lr = 0.0005

ori = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, add_data_dict, method='Uniform')
ori.performance(budget=0, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)

Method: Uniform, Budget: 0
======= Collect Data =======
[0 0 0 0 0 0 0 0 0 0]
======= Performance =======
Loss: 0.42593 (0.00428), Average EER: 0.21016 (0.00740), Max EER: 0.46287 (0.10862)



# System T Demo on Fashion-MNIST

## Use 3000 budget, lambda=0.1, "Moderate" strategy

In [6]:
budget = 3000
method = 'Moderate'

st = System_T((train_data, train_label), (val_data, val_label), val_data_dict, initial_data_array, num_class, add_data_dict)
st.selective_collect(budget=budget, k=10, batch_size=32, lr = lr, epochs=2000, cost_func=cost_func, 
                 Lambda=0.1, num_iter=5, slice_desc=slice_desc, strategy=method, show_figure=False)

======= Collect Data =======
[ 64  50  55 120 120  58 216  62  78  29]
Total Cost: 852, Remaining Budget: 2148
======= Collect Data =======
[  0  42 287 140 245  75 258  71 129   0]
Total Cost: 1247, Remaining Budget: 901
======= Collect Data =======
[379  59   0  73 117   5   0 108  62  98]
Total Cost: 901, Remaining Budget: 0

======= Performance =======
[443. 151. 342. 333. 482. 138. 474. 241. 269. 127.]
Number of iteration: 3
Strategy: Moderate, C: 0.1, Budget: 3000
Loss: 0.31776 (0.00430), Average EER: 0.15689 (0.00464), Max EER: 0.36049 (0.06239)



# Baseline: Uniform

In [7]:
budget = 3000
uni = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, add_data_dict, method='Uniform')
uni.performance(budget=budget, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)

Method: Uniform, Budget: 3000
======= Collect Data =======
[300 300 300 300 300 300 300 300 300 300]
======= Performance =======
Loss: 0.32214 (0.00503), Average EER: 0.17716 (0.00599), Max EER: 0.41855 (0.07153)



# Baseline: Water filling

In [8]:
budget = 3000
wf = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, add_data_dict, method='Waterfilling')
wf.performance(budget=budget, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)

Method: Waterfilling, Budget: 3000
======= Collect Data =======
[180 380 180 380 180 380 180 380 380 380]
======= Performance =======
Loss: 0.32970 (0.00678), Average EER: 0.20032 (0.00574), Max EER: 0.52133 (0.06015)



# Summary of results

<div align="center">
    
|<font size="5">Method</font>  | <font size="5"> Loss </font>| <font size="5"> Avg.EER </font>| 
|:---------------------:|:---------------------:|:---------------------:| 
|<font size=5> Original </font> | <font size=5> 0.42593  (&pm; 0.00214) </font> | <font size=5> 0.21016 (&pm; 0.00370)</font>|
|<font size=5> Uniform </font> | <font size=5> 0.32214  (&pm; 0.00251) </font> | <font size=5> 0.17716 (&pm; 0.00299)</font>|
|<font size=5> Water filling </font> | <font size=5> 0.32970  (&pm; 0.00339) </font> | <font size=5> 0.20032 (&pm; 0.00287)</font>|
|<font size=5> Moderate (ours) </font> | <font size=5> 0.31776  (&pm; 0.00215) </font> | <font size=5> 0.15689 (&pm; 0.00232)</font>|

</div>